# Entrenamiento del modelo

In [1]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import os

# Define las transformaciones para el conjunto de entrenamiento y validación
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = 'directorio_datos'  # Actualiza esto con la ruta a tu conjunto de datos
batch_size = 32

# Carga los conjuntos de datos
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=4)
               for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes


In [ ]:
from torchvision import models
from torch import nn

model = models.densenet201(pretrained=True)  # Utiliza el modelo preentrenado DenseNet-201

# Reemplaza la última capa clasificadora para que coincida con el número de clases del conjunto de datos
num_ftrs = model.classifier.in_features
model.classifier = nn.Linear(num_ftrs, 5)  # Ajuste para 5 clases
import torch

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

criterion = nn.CrossEntropyLoss()  # Define el criterio de pérdida
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)  # Usa el optimizador Adam


In [ ]:
num_epochs = 12  # Ajusta según sea necesario

for epoch in range(num_epochs):
    # Fases de entrenamiento y validación
    for phase in ['train', 'val']:
        if phase == 'train':
            model.train()  # Pone el modelo en modo de entrenamiento
        else:
            model.eval()   # Pone el modelo en modo de evaluación

        running_loss = 0.0
        running_corrects = 0

        # Iterar sobre los datos.
        for inputs, labels in dataloaders[phase]:
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            # forward
            # Rastrear el historial solo si estamos en fase de 'train'
            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)

                # backward + optimize solo si estamos en fase de 'train'
                if phase == 'train':
                    loss.backward()
                    optimizer.step()

            # Estadísticas
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / dataset_sizes[phase]
        epoch_acc = running_corrects.double() / dataset_sizes[phase]

        print(f'{phase} Epoch {epoch}/{num_epochs - 1}, Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}')

# Guarda el modelo

In [4]:
torch.save(model.state_dict(), 'densenet201_model.pth')


# Matriz de confusion y Recall

In [ ]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import confusion_matrix
import numpy as np
import os


# Transformaciones para las imágenes de prueba
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Cargar las imágenes de prueba
test_dir = 'directorio_datos_test'
test_dataset = datasets.ImageFolder(test_dir, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False, num_workers=4)
class_names = test_dataset.classes

# Función para realizar predicciones y calcular la matriz de confusión
def predict_and_confusion_matrix(model, dataloader):
    y_pred = []
    y_true = []
    model.eval()  # Establecer el modelo en modo de evaluación
    with torch.no_grad():  # No necesitamos calcular gradientes
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            y_pred.extend(preds.cpu().numpy())
            y_true.extend(labels.cpu().numpy())
    
    # Calcular la matriz de confusión
    conf_mat = confusion_matrix(y_true, y_pred)
    recall = recall_score(y_true, y_pred, average='macro') 
    return conf_mat, recall

# Calcular y mostrar la matriz de confusión
conf_mat, recall = predict_and_confusion_matrix(model, test_loader)
print(conf_mat)
print(f'Recall del modelo: {recall}')